In [1]:
import clang.cindex
import os
#파일 설정은 한번만 해야하는듯
clang.cindex.Config.set_library_file(r"C:\Program Files\LLVM\bin\libclang.dll")

In [2]:
#파싱
def parsing(file_path: str):
    index = clang.cindex.Index.create()
    return index.parse(file_path, args=['-std=c++14'])

In [3]:
def parse_project(directory):
    index = clang.cindex.Index.create()

    # 프로젝트 디렉토리 내의 모든 C++ 파일을 찾음
    files = list(find_cpp_files(directory))

    # 각 파일을 파싱
    translation_units = []
    for file in files:
        print(f"parse {file}")
        tu = index.parse(file, args=[
            '-x', 'c++', '-std=c++14',
            '-I', 'path/to/include',  # 추가 인클루드 경로
            '-D__CODE_GENERATOR__'   # 필요한 경우 매크로 정의
        ])
        translation_units.append(tu)

    return translation_units

def find_cpp_files(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(('.cpp', '.h')):
                yield os.path.join(root, file)

In [4]:
import time


folder_path = r"D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation"
# folder_path = r"D:\dev\AutoPlanning\trunk\AP_trunk_pure"
# Use the selected folder path for further processing
print("Selected folder:", folder_path)

start_time = time.time()
tus = parse_project(folder_path)

# 결과 처리 및 출력
for tu in tus:
    print(tu.spelling, "contains", len(list(tu.cursor.get_children())), "elements")

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")



Selected folder: D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorAxialFixture.cpp
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorAxialFixture.h
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorCrownDesign.cpp
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorCrownDesign.h
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorCSFixture.cpp
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorCSFixture.h
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorHybridFixture.cpp
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorHybridFixture.h
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorImplantPreview.cpp
parse D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\Actuat

In [5]:
def read_specific_column(filename, line_number, column_index):
    try:
        with open(filename, 'r') as file:
            for current_line_number, line in enumerate(file, start=1):
                if current_line_number == line_number:
                    if column_index <= len(line):
                        return line[:column_index-1] + "->" + line[column_index-1:]
                    else:
                        return "Error: The line does not have enough columns."
            return "Error: Line number out of range."
    except FileNotFoundError:
        return "Error: File not found."
    except Exception as e:
        return f"Error: {str(e)}"

def visit_node(node, depth=0):
    indent = ' ' * depth
    # print(f"{indent}{node.kind}: spelling='{node.spelling}', displayname='{node.displayname}', location={node.location.file.name}:{node.location.line}:{node.location.column}")
    print(f"{indent}{node.kind}: spelling='{node.spelling}', displayname='{node.displayname}',{node.location.line}:{node.location.column}")

    for child in node.get_children():
        visit_node(child, depth + 2)

def getSimpleFileName(input_path):
    # 경로를 표준화합니다.
    normalized_path = os.path.normpath(input_path)
    
    # 경로를 구성하는 파트들을 리스트로 분리합니다.
    parts = normalized_path.split(os.sep)
    
    # 마지막 파일 이름과 그 바로 상위 디렉토리만 포함하도록 조정합니다.
    if len(parts) > 1:
        result_path = os.path.join(parts[-2], parts[-1])
    else:
        result_path = parts[-1]  # 파일 또는 폴더만 있는 경우
    
    return result_path
    

In [6]:
class SimpleVisitor:
    def __init__(self, node, base_path: str):
        self.node: clang.cindex.Cursor = node
        self.base_path: str = base_path
        self.base_path_cpp: str = base_path.replace(".cpp", ".h")
        self.base_path_h: str = base_path.replace(".h", ".cpp")
        self.location: clang.cindex.SourceLocation = self.node.location
        self.child_node = self.node.get_children()

    def get_source_code(self):
        file = self.location.file
        if file:
            file_path: str = file.name
        else:
            file_path = self.base_path
        with open(file_path, 'r') as file:
            return file.read()
            
    def make_comment_code(self, code: str = None):
        if code is None:
            code = self.get_source_code()

        code2list = code.splitlines()
        line = self.location.line
        # 현재 노드에 대한 주석 추가
        try:
            code2list[line - 1] += f" // {self.node.kind.name}({self.node.spelling})"
        except:
            print(f"line={line} size = {code2list.__len__()}")
            return code
        code = '\n'.join(code2list)

        # 모든 자식 노드에 대해 재귀적으로 처리
        for child in self.node.get_children():
            child_visitor = SimpleVisitor(child, self.base_path)
            code = child_visitor.make_comment_code(code)

        return code




    def is_base_file(self):
        """
        재귀에서 node와 str을 입력 받음
        node 자체는 경로가 원래 상위의 경로가 아니니까
        :return:
        """
        location: clang.cindex.SourceLocation = self.node.location
        file: clang.cindex.File = location.file

        parsing_file_path: str = file.__str__()
        return parsing_file_path.__contains__(self.base_path) and parsing_file_path


    def visit(self):
        spelling = self.node.spelling
        location: clang.cindex.SourceLocation = self.node.location
        file: clang.cindex.File = location.file

        # print(f"node.base_path = {self.base_path}")

        parsing_file_path: str = file.__str__()
        # print("path : ", parsing_file_path)
        is_need_print = self.is_base_file()
        # print("->", is_need_print)
        if spelling and is_need_print:
            # self.node.kind의 이름을 출력하도록 수정
            print(f"Node found: {spelling} ({self.node.kind.name})")
            print(f"Node : {self.node}")
            print(f"Location : {location}")
            print(f"File : {file}")
            print()

        # 자식 노드에 대해 방문을 재귀적으로 진행
        for child in self.node.get_children():
            visitor = SimpleVisitor(child, self.base_path)
            visitor.visit()

    def for_make_file_map(self, map=dict()):
        spelling = self.node.spelling
        location: clang.cindex.SourceLocation = self.node.location
        file: clang.cindex.File = location.file
        str_file = file.__str__()
        if str_file in map:
            map[str_file].append(self.node)
        else:
            map[str_file] = [self.node]

        # 자식 노드에 대해 방문을 재귀적으로 진행
        for child in self.node.get_children():
            visitor = SimpleVisitor(child, self.base_path)
            visitor.for_make_file_map(map)

    def for_make_stmt_map(self, map):
        try:
            kind = self.node.kind
            if kind in map:
                map[kind].append(self.node)
            else:
                map[kind] = [self.node]

            for child in self.node.get_children():
                visitor = SimpleVisitor(child, self.base_path)
                visitor.for_make_stmt_map(map)

        except ValueError as e:
            print(f"Skipping node with unknown CursorKind: {e}")


    def for_make__map(self, map=dict()):
        spelling = self.node.spelling
        location: clang.cindex.SourceLocation = self.node.location
        file: clang.cindex.File = location.file
        str_file = file.__str__()
        if str_file in map:
            map[str_file].append(self.node)
        else:
            map[str_file] = [self.node]

        # 자식 노드에 대해 방문을 재귀적으로 진행
        for child in self.node.get_children():
            visitor = SimpleVisitor(child, self.base_path)
            visitor.for_make_file_map(map)
    def compare_spelling_display(self):
        if self.node.spelling != self.node.displayname:
            print(f"{node.kind}: spelling='{node.spelling}', displayname='{node.displayname}', location={node.location.file}:{node.location.line}:{node.location.column}")
        for chid_node in self.node.get_children():
            visitor = SimpleVisitor(chid_node, self.base_path)
            visitor.compare_spelling_display()
            

    def visit_for_location(self):
        # print(type(self.node))
        spelling = self.node.spelling
        location: clang.cindex.SourceLocation = self.node.location
        file = location.file
        print("file : ", type(file), " = ", file)

        print(f"{type(location)} = {location}")
        tokens = self.node.get_tokens()
        print(tokens)

        if spelling:  # 노드 이름이 있는 경우에만 출력
            location = self.node.location
            line = location.line
            column = location.column
            print(f"Node found: {spelling} ({self.node.kind.name}) at Line: {line}, Column: {column}")

        # 자식 노드에 대해 방문을 재귀적으로 진행
        for child in self.node.get_children():
            visitor = SimpleVisitor(child)
            visitor.visit_for_location()


In [7]:
#모든 파일( CompliationUnit) 순환
i =0
for tunit in tus:
    print(f"{[i]}={tunit.spelling}")
    i += 1

#특정 파일 설정
target = tus[6]

#Visitor 생성 및 stmt 파싱
simpleVisit = SimpleVisitor(target.cursor, target.spelling)
stmt_map = {}
simpleVisit.for_make_stmt_map(stmt_map)
print(target.spelling, " has ")
for stmt in stmt_map:
    print(stmt, " : ", stmt_map[stmt].__len__())
print("done")

[0]=D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorAxialFixture.cpp
[1]=D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorAxialFixture.h
[2]=D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorCrownDesign.cpp
[3]=D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorCrownDesign.h
[4]=D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorCSFixture.cpp
[5]=D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorCSFixture.h
[6]=D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorHybridFixture.cpp
[7]=D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorHybridFixture.h
[8]=D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorImplantPreview.cpp
[9]=D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorImplantPreview.h
[10]=D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorMPRFi

In [ ]:
#라인별 map 출력
line_map = mycursor.get_visit_line_map()
for num in line_map:
    print(num, line_map[num])

In [79]:
from clang.cindex import CursorKind
from clang.cindex import Cursor as ClangCursor
CXX_METHOD_list: ClangCursor = stmt_map[CursorKind.CXX_METHOD]

planning_method_list= [ClangCursor]
#메서드 선언도 파일별 분리가 필요함
for node in CXX_METHOD_list:
    if node.location.file.name.__contains__("AutoPlanning"):
        planning_method_list.append(node)
        print(f"{node.displayname} {getSimpleFileName(node.location.file.name)} : {node.location.line}")



operator=(const float *) inlines\usefulinlines.h : 42
operator=(const float3 &) inlines\usefulinlines.h : 47
operator+(const float3 &) inlines\usefulinlines.h : 52
operator-(const float3 &) inlines\usefulinlines.h : 57
operator-() inlines\usefulinlines.h : 62
operator*(const float3 &) inlines\usefulinlines.h : 67
operator*(const float) inlines\usefulinlines.h : 72
operator/(const float3 &) inlines\usefulinlines.h : 77
operator/(const float) inlines\usefulinlines.h : 82
operator[](int) inlines\usefulinlines.h : 87
operator==(const float3 &) inlines\usefulinlines.h : 92
operator!=(const float3 &) inlines\usefulinlines.h : 99
Magnitude() inlines\usefulinlines.h : 103
Normalize() inlines\usefulinlines.h : 107
GetDistance(const float3 &, const float3 &) inlines\usefulinlines.h : 123
CrossProduct(const float3 &, const float3 &) inlines\usefulinlines.h : 129
DotProduct(const float3 &, const float3 &) inlines\usefulinlines.h : 139
CalculateVectorsRadianAngle(const float3 &, const float3 &) inl

In [89]:
#호출 식 외부 파일 필터링 (cpp 파일에서)
call_exp_node_list:[clang.cindex.Cursor] = stmt_map[clang.cindex.CursorKind.CALL_EXPR      ] 
print("타겟 파일 : ", target.spelling)
print(f"외부 파일 포함 {call_exp_node_list.__len__()}")

file_set = set()
file_list = []

temp:[clang.cindex.Cursor] = []
for node in call_exp_node_list:
    if node.location.file.name.__contains__("AutoPlanning"):
        temp.append(node)
        file_set.add(node.location.file.name)
        file_list.append(node)
        
call_exp_node_list = temp

print(f"내부 파일 {call_exp_node_list.__len__()}")

# print("파일 종류")
# for fname in file_set:
#     print(fname) #../balbal/../blabla  와 같은 헤더 --> 헤더 형식으로도 들어오는듯


print("Call EXPR 순환")

#내부 파일에 포함된 CallExpr 리스트 순환
check_set = set()
for node in file_list:
    if check_set.__contains__(node.location.file.name):
        continue
    else:
        check_set.add(node.location.file.name)
        print(f"{node.displayname} {getSimpleFileName(node.location.file.name)} : {node.location.line}")

타겟 파일 :  D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorHybridFixture.cpp
외부 파일 포함 31081
내부 파일 2154
Call EXPR 순환
float3 inlines\usefulinlines.h : 54
operator[] BaseTools\Vector.h : 68
Reset BaseTools\tinyxml2.h : 155
operator= AppFrame\SurfaceLoader.h : 47
 AppFrame\SurfaceModelDoc.h : 83
operator= AppFrame\ToothDoc.h : 334
CStringT AppFrame\SleeveDoc.h : 35
CStringT AppFrame\AbutmentDoc.h : 28
operator= AppFrame\LocalDocBase.h : 265
operator= AppFrame\ModelData.h : 12
operator= AppFrame\SurgicalGuideData.h : 19
CStringT AppFrame\SinusLiftData.h : 13
float3 DoitPrimitive\CoordinateConverter.h : 74
CRect AppFrame\DentalArchData.h : 132
CMatrixMxN AppFrame\PatchLineSurgicalGuideData.h : 19
operator= AppFrame\LineTrimmingData.h : 24
CMatrixMxN AppFrame\BiteAlignmentData.h : 20
CMatrixMxN AppFrame\NamingBarData.h : 21
CMatrixMxN AppFrame\IDTagData.h : 28
float3 BaseTools\CardinalSpline.h : 111
float3 AppFrame\NerveData.h : 62
operator= AppFrame\OcclusalPlaneData.h 

In [62]:
#cpp File 커서에서 특정 구문 전체 검색 (클래스 검색?) 

cpp_cursor:clang.cindex.Cursor  = target.cursor
cppVisitor = SimpleVisitor(cpp_cursor, target.spelling)
stmt_map = {}
simpleVisit.for_make_stmt_map(stmt_map)


memberRefExprs = stmt_map[clang.cindex.CursorKind.MEMBER_REF_EXPR]
name_set = set()
temp = []
updateMSG=[]

for node in memberRefExprs:
    if node.spelling.__contains__("UpdateMessage"):
        updateMSG.append(node)
    
    if node.location.file and node.location.file.name.__contains__("AutoPlanning"):
        temp.append(node)
        # print(f"{node.displayname} {node.location.file.name}")
        name_set.add(node.displayname)
memberRefExprs = temp
#계층 구조
# visit_node(update_node)

print(updateMSG.__len__())
for node in updateMSG:
    print(f"{node.displayname} {node.location}")
# for name in name_set:
    # print(name)



Skipping node with unknown CursorKind: Unknown template argument kind 280
12
PostUpdateMessage 1280
PostUpdateMessage 1284
SendUpdateMessage 2578
PostUpdateMessage 2882
SendUpdateMessage 3051
SendUpdateMessage 3074
SendUpdateMessage 3078
PostUpdateMessage 3741
PostUpdateMessage 3814
SendUpdateMessage 3841
SendUpdateMessage 3856
PostUpdateMessage 3890


In [43]:
#cpp File 커서에서 특정 메서드 커서로 
target_cursor:clang.cindex.Cursor  = target.cursor
for child in target_cursor.get_children():
    if child.kind.name == "CXX_METHOD" and child.spelling == "OnUpdate":
        update_node = child
        break

#특정 메서드 분석
update_visitor = SimpleVisitor(update_node, update_node.location.file.name)
stmt_map={}
update_visitor.for_make_stmt_map(stmt_map)
for stmt_key in stmt_map:
    print(f"{stmt_key} : {stmt_map[stmt_key].__len__()}")

#계층 구조
# visit_node(update_node)





CursorKind.CXX_METHOD : 1
CursorKind.TYPE_REF : 75
CursorKind.PARM_DECL : 4
CursorKind.COMPOUND_STMT : 60
CursorKind.DECL_STMT : 52
CursorKind.VAR_DECL : 52
CursorKind.UNEXPOSED_EXPR : 564
CursorKind.MEMBER_REF_EXPR : 343
CursorKind.DECL_REF_EXPR : 500
CursorKind.CXX_DYNAMIC_CAST_EXPR : 3
CursorKind.STRING_LITERAL : 120
CursorKind.CSTYLE_CAST_EXPR : 39
CursorKind.IF_STMT : 82
CursorKind.BINARY_OPERATOR : 82
CursorKind.NAMESPACE_REF : 42
CursorKind.CALL_EXPR : 291
CursorKind.OVERLOADED_DECL_REF : 200
CursorKind.CXX_REINTERPRET_CAST_EXPR : 3
CursorKind.CXX_BOOL_LITERAL_EXPR : 53
CursorKind.INTEGER_LITERAL : 31
CursorKind.PAREN_EXPR : 7
CursorKind.FOR_STMT : 2
CursorKind.UNARY_OPERATOR : 5
CursorKind.CXX_STATIC_CAST_EXPR : 4
CursorKind.CXX_THIS_EXPR : 3
CursorKind.WHILE_STMT : 1
CursorKind.NULL_STMT : 1
CursorKind.RETURN_STMT : 1


In [44]:
call_exp_node_list:[clang.cindex.Cursor] = stmt_map[clang.cindex.CursorKind.CALL_EXPR      ] 
print(f"외부 파일 포함 {call_exp_node_list.__len__()}")

print(target.spelling)

file_set = set()

file_list = []

temp:[clang.cindex.Cursor] = []
for node in call_exp_node_list:
    if node.location.file.name.__contains__("AutoPlanning"):
        temp.append(node)
        file_set.add(node.location.file.name)
        file_list.append(node)
call_exp_node_list = temp
print(f"내부 파일 {call_exp_node_list.__len__()}")
for fname in file_set:
    print(fname)


check_set = set()
for node in file_list:
    if check_set.__contains__(node.location.file.name):
        continue
    else:
        check_set.add(node.location.file.name)
        print(f"{node.displayname} {node.location}")

외부 파일 포함 291
D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorHybridFixture.cpp
내부 파일 291
D:\dev\AutoPlanning\trunk\AP_trunk_pure\mod_APImplantSimulation\ActuatorHybridFixture.cpp
 <SourceLocation file 'D:\\dev\\AutoPlanning\\trunk\\AP_trunk_pure\\mod_APImplantSimulation\\ActuatorHybridFixture.cpp', line 332, column 8>


In [8]:
for expNode in stmt_map[clang.cindex.CursorKind.MEMBER_REF_EXPR   ]:
    if expNode.location.file:
        simple_file_name = getSimpleFileName(expNode.location.file.name)
    
    print(f"Node found: {expNode.spelling} ({expNode.kind.name})")
    # print(f"Location : {expNode.location}")
    print(f"Location :{simple_file_name} {expNode.location.line} : {expNode.location.column}") #(파일은 동일한듯?)

In [59]:
#1차 자식만 탐색
root_node: clang.cindex.Cursor = target.cursor


other_file_nodes = []
nodes = []

for node in root_node.get_children():
    node_file: clang.cindex.File = node.location.file

    if node_file and node_file.name == target.spelling:
        nodes.append(node)
    else:
        other_file_nodes.append(node)

for node in nodes:
    print(f"Node found: {node.spelling} ({node.kind.name})")
    print(f"Location : {node.location}")



print(f"other size : {other_file_nodes.__len__()} size : {nodes.__len__()}")

code = None
for node in nodes:
    simpleVisit = SimpleVisitor(node, target.spelling)
    code = simpleVisit.make_comment_code(code)

# print(code)

# 파일을 쓰기 모드로 열기 (파일이 없다면 생성됩니다)
with open('result.cpp', 'w') as file:
    file.write(code)



source = simpleVisit.make_comment_code()
print(target.spelling)

#많이 나온다.. 파일 필터링은 필수.
# for node in other_file_nodes:
#     if node.kind == clang.cindex.CursorKind.CXX_METHOD or node.kind == clang.cindex.CursorKind.CONSTRUCTOR:
#         print(f"Node found: {node.spelling} ({node.kind.name})")
#         print(f"Location : {node.location}")


Node found: CActuatorPanoFixture (CONSTRUCTOR)
Location : <SourceLocation file 'D:\\dev\\AutoPlanning\\trunk\\AP_trunk_pure\\mod_APImplantSimulation\\ActuatorPanoFixture.cpp', line 31, column 23>
Node found: ~CActuatorPanoFixture (DESTRUCTOR)
Location : <SourceLocation file 'D:\\dev\\AutoPlanning\\trunk\\AP_trunk_pure\\mod_APImplantSimulation\\ActuatorPanoFixture.cpp', line 39, column 23>
Node found: OnUpdate (CXX_METHOD)
Location : <SourceLocation file 'D:\\dev\\AutoPlanning\\trunk\\AP_trunk_pure\\mod_APImplantSimulation\\ActuatorPanoFixture.cpp', line 50, column 28>
Node found: OnInitialUpdate (CXX_METHOD)
Location : <SourceLocation file 'D:\\dev\\AutoPlanning\\trunk\\AP_trunk_pure\\mod_APImplantSimulation\\ActuatorPanoFixture.cpp', line 184, column 28>
Node found: OnActLBDown (CXX_METHOD)
Location : <SourceLocation file 'D:\\dev\\AutoPlanning\\trunk\\AP_trunk_pure\\mod_APImplantSimulation\\ActuatorPanoFixture.cpp', line 216, column 28>
Node found: OnActMouseMove (CXX_METHOD)
Locatio

In [9]:
# source_code = simpleVisit.get_source_code()

# #print(source_code)
# print("========")

# source_code = simpleVisit.make_comment_code()
# print(source_code)

In [10]:
# dataMap:dict[str, list] = {}

# cpp_list = []
# simpleVisit.for_make_file_map(dataMap)
# for key in dataMap:
#     print(f"{key} : {dataMap[key].__len__()}")
#     if key.__contains__(".cpp"):
#         cpp_list.append((key, dataMap[key]))

# print("=================")

# for key, data in cpp_list:
#     print(f"{key} : {data.__len__()}")
#     for node in data:
#         location: clang.cindex.SourceLocation = node.location
#         source = read_specific_column(key, location.line, location.column)
#         print("====")
#         print(f"{source}[{location.line}:{location.column}]{node.spelling} {node.kind}" )
#         print("====")


# stmt_map = {}
# simpleVisit.for_make_stmt_map(stmt_map)

# for stmt in stmt_map:
#     print(f"{stmt.name} {stmt_map[stmt].__len__()}")

# for node in stmt_map[clang.cindex.CursorKind.CALL_EXPR]:
#     location: clang.cindex.SourceLocation = node.location

#     file_name: str = location.file.name
#     if file_name.__contains__(target.spelling):
#         print(f"{file_name} contains {target.spelling}")
#         source = read_specific_column(location.file.name, location.line, location.column)
#         print("=================")
#         print(f"{source}[{location.line}:{location.column}]{node.spelling} {node.kind}")

